In [0]:
from Helpers import *
  
sc = SparkContext.getOrCreate()

## Input Parameters

In [0]:
# Get Input Parameters
print(f"AWS_REGION: {AWS_REGION}")
print(f"LOAD_DATE: {LOAD_DATE}")
print(f"STATUS_TABLE: {STATUS_TABLE}")
print(f"SNS_TOPIC: {SNS_TOPIC}")
print(f"RWE_BUCKET: {RWE_BUCKET}")
print(f"UNPROCESSED_PATH: {UNPROCESSED_PATH}")

In [0]:
status_handler = SuperiorProcessStatusHandler(AWS_REGION, STATUS_TABLE, SNS_TOPIC, LOAD_DATE)
pending_batches = status_handler.get_items_with_status('Pending', LOAD_DATE)
processed_batches = status_handler.get_items_with_status('Processed', LOAD_DATE)

print(f"The number of Pending Batches: {len(pending_batches)}")
print(f"The number of Processed Batches: {len(processed_batches)}")

## Unprocessed Record Count

In [0]:
# If there are no unprocessed records, this will throw an AnalysisException
try:
    unprocessed_measurements_df = spark.read.parquet(f'{UNPROCESSED_PATH}/measurement/{LOAD_DATE}/*')
    unprocessed_measurement_count = unprocessed_measurements_df.count()
except AnalysisException as ae:
    print(ae)
    unprocessed_measurement_count = 0

In [0]:
# If there are no unprocessed records, this will throw an AnalysisException
try:
    unprocessed_devices_df = spark.read.parquet(f'{UNPROCESSED_PATH}/device/{LOAD_DATE}/*')
    unprocessed_devices_count = unprocessed_devices_df.count()
except AnalysisException as ae:
    print(ae)
    unprocessed_devices_count = 0

In [0]:
print(f"The number of Unprocessed Measurement: {unprocessed_measurement_count}")
print(f"The number of Unprocessed Devices: {unprocessed_devices_count}")

## Notify

In [0]:
# Notify Processed/Pending Batches & Unprocessed Measurements/Devices
sns_notifier = SNSNotifier(AWS_REGION, SNS_TOPIC, LOAD_DATE)
sns_notifier.notify_status(len(pending_batches), len(processed_batches), unprocessed_measurement_count, unprocessed_devices_count)